In [1]:
from kafka import KafkaConsumer
import json

In [2]:
KAFKA_HOST = "kafka"
KAFKA_PORT = 9092
KAFKA_VERSION = (0, 10, 2)
BOOTSTRAP_SERVER = f"{KAFKA_HOST}:{KAFKA_PORT}"

In [5]:
consumer = KafkaConsumer(
 bootstrap_servers=[BOOTSTRAP_SERVER],
 security_protocol="PLAINTEXT",
 value_deserializer = lambda v: json.loads(v.decode('ascii')),
 auto_offset_reset='earliest'
)

In [ ]:
consumer.subscribe(topics='hotel-booking-request')
for message in consumer:
    print(f"{message.partition}: {message.offset}: v={message.value}")
    

0: 1: v={'name': 'Carlo Bianchi', 'hotel': 'Average Hotel', 'dateFrom': '12-07-2021', 'dateTo': '23-07-2021', 'details': 'Room next to the highway 🚗🚗🚗🚗'}
0: 2: v={'name': 'Giuseppe Rossi', 'hotel': 'Luxury Hotel', 'dateFrom': '25-06-2021', 'dateTo': '07-07-2021', 'details': 'I want the best room 😀😀😀😀😀!!!!'}
0: 3: v={'name': 'Carlo Bianchi', 'hotel': 'Average Hotel', 'dateFrom': '12-07-2021', 'dateTo': '23-07-2021', 'details': 'Room next to the highway 🚗🚗🚗🚗'}
